In [1]:
import sys
import os
os.chdir("..")

from os.path import dirname

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn import preprocessing

from sail.models.torch.os_cnn import OS_CNN_CLassifier

/home/abduvoris/anaconda3/envs/imla_assignment/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def set_nan_to_zero(a):
    where_are_NaNs = np.isnan(a)
    a[where_are_NaNs] = 0
    return a

def TSC_data_loader(dataset_path,dataset_name):
    Train_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TRAIN.tsv')
    Test_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TEST.tsv')
    Train_dataset = Train_dataset.astype(np.float32)
    Test_dataset = Test_dataset.astype(np.float32)

    X_train = Train_dataset[:, 1:]
    y_train = Train_dataset[:, 0:1]

    X_test = Test_dataset[:, 1:]
    y_test = Test_dataset[:, 0:1]
    le = preprocessing.LabelEncoder()
    le.fit(np.squeeze(y_train, axis=1))
    y_train = le.transform(np.squeeze(y_train, axis=1))
    y_test = le.transform(np.squeeze(y_test, axis=1))
    return set_nan_to_zero(X_train), y_train, set_nan_to_zero(X_test), y_test


def load_to_torch(X_train, y_train, X_test, y_test, device):
    X_train = torch.from_numpy(X_train)
    X_train.requires_grad = False
    X_train = X_train.to(device)
    y_train = torch.from_numpy(y_train).to(device)

    X_test = torch.from_numpy(X_test)
    X_test.requires_grad = False
    X_test = X_test.to(device)
    y_test = torch.from_numpy(y_test).to(device)


    if len(X_train.shape) == 2:
        X_train = X_train.unsqueeze_(1)
        X_test = X_test.unsqueeze_(1)
    return X_train, y_train, X_test, y_test

In [4]:
dataset_path = dirname("./notebooks/OS_CNN/UCRArchive_2018/")
dataset_name = "FiftyWords"

# load data,
X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)
print('train data shape', X_train.shape)
print()
print('train label shape',y_train.shape)
print('test data shape',X_test.shape)
print('test label shape',y_test.shape)
print('unique train label',np.unique(y_train))
print('unique test label',np.unique(y_test))
device = "cpu"
X_train, y_train, X_test, y_test = load_to_torch(X_train, y_train, X_test, y_test, device)

Max_kernel_size = 89
start_kernel_size = 1
input_channel = X_train.shape[1] # input channel size
n_class = max(y_train) + 1 # output class number
receptive_field_shape= min(int(X_train.shape[-1]/4),Max_kernel_size) # receptive fields

model = OS_CNN_CLassifier(n_class.item(), input_channel, receptive_field_shape)
model.fit(X_train, y_train)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 16
N_test=len(test_dataset)
train_loader = DataLoader(train_dataset, batch_size=max(int(min(X_train.shape[0] / 10, batch_size)),2), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=max(int(min(X_train.shape[0] / 10, batch_size)),2), shuffle=False)

train data shape (450, 270)

train label shape (450,)
test data shape (455, 270)
test label shape (455,)
unique train label [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
unique test label [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]


/home/abduvoris/anaconda3/envs/imla_assignment/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.6838       0.2222        3.2757  1.5532
      2        3.1325       0.2222        2.9934  1.6128
      3        2.8331       0.2444        2.6731  1.5441
      4        2.5332       0.3444        2.4476  1.5825
      5        2.2659       0.3111        2.3056  1.5683
      6        2.0388       0.4111        1.9912  1.5885
      7        1.8238       0.5333        1.8264  1.7760
      8        1.6135       0.5222        1.7733  1.7264
      9        1.4831       0.6333        1.5817  1.6401
     10        1.3077       0.5667        1.6581  1.6337
     11        1.1867       0.5778        1.7216  1.5677
     12        1.0624       0.5889        1.5658  1.5669
     13        0.8965       0.6000        1.3917  1.6197
     14        0.7146       0.6222        1.4322  1.7137
     15        0.5869       0.6333        1.4565  1.7100
     16        0.4583       0.6

In [5]:
accuracy_list=[]
correct = 0
N_test=len(test_dataset)
yhat = model.predict(X_test)
correct += (torch.tensor(yhat) == y_test).sum().item()
accuracy = correct / N_test
accuracy

0.7164835164835165

In [ ]:
# # partial fit

# cnt = 0
# wait_samples = 10 


# model = OS_CNN_CLassifier(n_class.item(), input_channel, receptive_field_shape)

# for sample in train_loader:


#     model.partial_fit(sample[0], sample[1])
#     y_hat = model.predict


#     # Test every n samples
#     if (cnt % wait_samples == 0) & (cnt != 0):
#         model
#     y_predict = model(sample[0]) # prediction
#     output = lossfunc(y_predict, sample[1]) # calculate loss
#     output.backward() # backpropagation, compute gradients
#     optimizer.step() # apply gradients
#     COST += output.data

# #     cost_list.append(COST)
# #     correct = 0
# #     scheduler.step(output)
# #     # perform prediction on the validation set
# #     for x_test, y_test in test_loader:
# #         z = model(x_test)
# #         _, yhat = torch.max(z.data, 1)
# #         correct += (yhat == y_test).sum().item()
# #     accuracy = correct / N_test
# #     accuracy_list.append(accuracy)

In [6]:
# # Plot the loss and accuracy
# import matplotlib.pyplot as plt

# fig, ax1 = plt.subplots()
# color = 'tab:red'
# ax1.plot(cost_list, color=color)
# ax1.set_xlabel('epoch', color=color)
# ax1.set_ylabel('Cost', color=color)
# ax1.tick_params(axis='y', color=color)
    
# ax2 = ax1.twinx()  
# color = 'tab:blue'
# ax2.set_ylabel('accuracy', color=color) 
# ax2.set_xlabel('epoch', color=color)
# ax2.plot( accuracy_list, color=color)
# ax2.tick_params(axis='y', color=color)
# fig.tight_layout()